# Instant Gratification!

Let's build a useful LLM solution - in a matter of minutes.

Our goal is to code a new kind of Web Browser. Give it a URL, and it will respond with a summary. The Reader's Digest of the internet!!

Before starting, be sure to have followed the instructions in the "README" file, including creating your API key with OpenAI and adding it to the `.env` file.

## If you're new to Jupyter Lab

Welcome to the wonderful world of Data Science experimentation! Once you've used Jupyter Lab, you'll wonder how you ever lived without it. Simply click in each "cell" with code in it, like the cell immediately below this text, and hit Shift+Return to execute that cell. As you wish, you can add a cell with the + button in the toolbar, and print values of variables, or try out variations.

If you need to start again, go to Kernel menu >> Restart kernel.

In [3]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# Connecting to OpenAI

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.

## Troubleshooting if you have problems:

1. OpenAI takes a few minutes to register after you set up an account. If you receive an error about being over quota, try waiting a few minutes and try again.
2. Also, double check you have the right kind of API token with the right permissions. You should find it on [this webpage](https://platform.openai.com/api-keys) and it should show with Permissions of "All". If not, try creating another key by:
- Pressing "Create new secret key" on the top right
- Select **Owned by:** you, **Project:** Default project, **Permissions:** All
- Click Create secret key, and use that new key in the code and the `.env` file (it might take a few minutes to activate)
- Do a Kernel >> Restart kernel, and execute the cells in this Jupyter lab starting at the top
4. As a fallback, replace the line `openai = OpenAI()` with `openai = OpenAI(api_key="your-key-here")` - while it's not recommended to hard code tokens in Jupyter lab, because then you can't share your lab with others, it's a workaround for now
5. Contact me! Message me or email ed@edwarddonner.com and we will get this to work.

Any concerns about API costs? See my notes in the README - costs should be minimal, and you can control it at every point.

In [4]:
# Load environment variables in a file called .env

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY','your-key-if-not-using-env')
openai = OpenAI()

In [5]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [6]:
# Let's try one out

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connect
with me for

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [7]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [8]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

In [9]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

## Time to bring it together - the API for OpenAI is very simple!

In [10]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [11]:
summarize("https://edwarddonner.com")

'# Summary of Edward Donner\'s Website\n\nThe website is a personal platform for Ed Donner, who is passionate about coding and experimenting with large language models (LLMs). He is the co-founder and CTO of Nebula.io, which focuses on applying AI to enhance talent discovery and management. Prior to this, he founded an AI startup called untapt that was acquired in 2021. \n\nEd shares his interests in DJing, amateur electronic music production, and engaging with technical discussions on Hacker News. His work has garnered notable achievements, including patented matching models and positive press coverage.\n\n## Recent Posts\n1. **Mastering AI and LLM Engineering – Resources** (November 13, 2024)\n2. **From Software Engineer to AI Data Scientist – Resources** (October 16, 2024)\n3. **Outsmart LLM Arena – A Battle of Diplomacy and Deviousness** (August 6, 2024)\n4. **Choosing the Right LLM: Toolkit and Resources** (June 26, 2024)\n\nThe website also features a section titled "Outsmart," w

In [13]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [14]:
display_summary("https://edwarddonner.com")

# Summary of Edward Donner's Website

The website serves as a personal platform for Edward Donner, a software engineer and AI enthusiast. He is the co-founder and CTO of Nebula.io, where he focuses on leveraging AI to improve talent discovery and engagement. The site reflects his interests in coding, LLMs (Large Language Models), electronic music, and tech discussions on platforms like Hacker News.

## Key Announcements and Posts
- **November 13, 2024**: Post on mastering AI and LLM engineering with resources.
- **October 16, 2024**: Insights on transitioning from software engineer to AI data scientist.
- **August 6, 2024**: Introduction to the "Outsmart LLM Arena," focusing on competitive LLM strategies.
- **June 26, 2024**: Discussion on how to choose the right LLM, with accompanying tools and resources.

Overall, the site serves as a blend of personal branding, technology insights, and professional development advice in the AI field.

In [15]:
display_summary("https://cnn.com")

# CNN Website Summary

CNN is a major news outlet providing breaking news, latest stories, and video content covering various topics including:

- **US and International News**: Updates on significant events both domestically and worldwide.
- **Politics**: Current political developments, particularly focusing on elections and legislation.
- **Health**: Information related to public health, wellness, and medical advancements.
- **Entertainment**: News on movies, celebrities, and cultural events.
- **Sports**: Coverage of various sports including football, basketball, and more.
- **Science and Climate**: Insights into scientific discoveries and environmental issues.

### Recent News Highlights

1. **Christmas Market Attack in Germany**: A driver plowed into a crowd at a Christmas market in Magdeburg, resulting in at least two fatalities and multiple injuries. The driver has been arrested and is reportedly from Saudi Arabia.

2. **Potential US Government Shutdown**: US House Speaker Johnson announced that a government shutdown will be averted, though specific details of the plan remain unclear.

3. **Ukraine-Russia Conflict**: A recent missile strike by Russia in Kyiv resulted in casualties and damage to embassies, as described by Ukrainian officials.

4. **Cultural and Scientific Discoveries**: A notable discovery in Germany could alter the understanding of Christian history in Europe, and 27 new species have been discovered in Peru, showcasing the ongoing exploration of biodiversity.

These summaries provide insight into the type of content available on CNN, highlighting its commitment to delivering timely and relevant news across a broad spectrum of topics.

In [16]:
display_summary("https://anthropic.com")

# Summary of Anthropic Website

Anthropic is an AI safety and research company based in San Francisco, focusing on developing reliable and beneficial AI systems. The company has introduced its latest AI model, **Claude 3.5 Sonnet**, which is described as their most intelligent AI model to date. They also announced **Claude 3.5 Haiku** as part of their ongoing model updates.

## Recent Announcements
- **October 22, 2024**: Introduction of new functionalities including computer use and the release of Claude 3.5 Sonnet and Claude 3.5 Haiku.
- **September 4, 2024**: Announcement related to Claude for Enterprise.
- **March 8, 2023**: Release of "Core Views on AI Safety," discussing the principles behind AI safety and alignment.
- **December 15, 2022**: Publication of research on "Constitutional AI," focusing on generating harmlessness from AI feedback.

Overall, the website offers insights into Anthropic’s work in AI research, product development, and its commitment to safety in AI technologies. Users can explore options to build applications using their API and are encouraged to check open career opportunities within the company.

## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them. Please push your code afterwards so I can share it with other students!

In [17]:
display_summary("https://openai.com")

# Website Summary

The website currently displays a message indicating that JavaScript and cookies need to be enabled for proper functionality. As there is no additional content available without these features activated, a detailed summary of the site's offerings, including news or announcements, cannot be provided at this time.

In [2]:
%pip install -U selenium

Note: you may need to restart the kernel to use updated packages.


In [17]:
#Parse webpages which is designed using JavaScript heavely
# download the chorme driver from here as per your version of chrome - https://developer.chrome.com/docs/chromedriver/downloads
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

#PATH_TO_CHROME_DRIVER = 'C:\\Users\\sahab\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromrdriver.exe'

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url

        options = Options()

        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")

        #service = Service(PATH_TO_CHROME_DRIVER)
        service = Service("C:/Users/sahab//Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe")
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        input("Please complete the verification in the browser and press Enter to continue...")
        page_source = driver.page_source
        driver.quit()

        soup = BeautifulSoup(page_source, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.get_text(separator="\n", strip=True)

In [20]:
#Parse webpages which is designed using JavaScript heavely
# download the chorme driver from here as per your version of chrome - https://developer.chrome.com/docs/chromedriver/downloads
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

#PATH_TO_CHROME_DRIVER = 'C:\\Users\\sahab\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromrdriver.exe'

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url

        options = Options()

        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        
        # Get chromedriver path dynamically
        chromedriver_path = os.getenv("CHROMEDRIVER_PATH", "C:/Users/sahab//Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe")
        service = Service(chromedriver_path)

        try:
            # Launch the browser and navigate to the URL
            with webdriver.Chrome(service=service, options=options) as driver:
                driver.get(url)

                # Wait for manual CAPTCHA verification
                input("Please complete the verification in the browser and press Enter to continue...")

                # Get the page source
                page_source = driver.page_source

        except Exception as e:
            print(f"Error occurred while processing the webpage: {e}")
            return

        soup = BeautifulSoup(page_source, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.get_text(separator="\n", strip=True)

In [ ]:
display_summary("https://openai.com")

In [22]:
display_summary("https://edwarddonner.com")

Please complete the verification in the browser and press Enter to continue... 


# Summary of Edward Donner's Website

The website showcases Edward Donner, a co-founder and CTO of Nebula.io, with a focus on AI and LLM (Large Language Model) technologies. Ed shares his passion for coding, LLMs, and amateur music production. He previously led an AI startup, untapt, before its acquisition in 2021.

## Key Sections

- **Outsmart**: Describes an arena where LLMs compete in a "battle of diplomacy and deviousness."
- **Recent Posts**:
  - **November 13, 2024**: *Mastering AI and LLM Engineering – Resources*
  - **October 16, 2024**: *From Software Engineer to AI Data Scientist – Resources*
  - **August 6, 2024**: *Outsmart LLM Arena – A Battle of Diplomacy and Deviousness*
  - **June 26, 2024**: *Choosing the Right LLM: Toolkit and Resources*

Overall, the website highlights Ed's work and interests in AI, providing resources and insights related to LLMs, technology, and personal development in the field.

In [23]:
display_summary("https://cnn.com")

Please complete the verification in the browser and press Enter to continue... 


# CNN Website Summary

CNN is a news organization providing breaking news, articles, and videos across various topics including US and world news, politics, business, health, entertainment, and more. The website features live updates on significant global events and major incidents, such as ongoing conflicts (e.g., the Ukraine-Russia War and the Israel-Hamas War).

### Key Recent News Highlights:
- **German Christmas Market Attack**: A driver crashed into a crowd in Magdeburg, resulting in dozens of injuries and at least two fatalities. The suspect has been arrested and is reported to be from Saudi Arabia.
- **US Government Shutdown**: The House passed a spending package to prevent a government shutdown just before the deadline, pending Senate approval.
- **Conflict in Kyiv**: Ukraine reports casualties and damage to embassies from a recent Russian missile barrage.

Additional notable stories include:
- Recent archaeological discoveries that could impact European Christian history.
- Elon Musk's political endorsements and comments on international affairs.
- Ongoing labor actions among Amazon drivers amidst the holiday season.

The site emphasizes its commitment to delivering timely and comprehensive news coverage across a range of sectors, allowing users to stay informed on both domestic and international issues.